# PyTorch Lightning
* https://www.youtube.com/watch?v=Hgg8Xy6IRig&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=21
* Thanks to Lightinig, we don't have to consider following things anymore
  1. model.train(), model.eval()
  2. code for gpu
    * device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') , model.to(device)が不要になる
    * easy GPU/TPU support
    * scale GPUS
  3. 学習プロセスにおける以下のコードも考慮不要
    * optimizer.zero_grad()
    * loss.backward()
    * optimizer.step()
  4. 予測時の以下コードも考慮不要
    * with torch.no_grad():

In [ ]:
!pip install pytorch-lightning

* Y5_feed_forwad_Neural_Network.ipynb のコードをLightningに書き換える

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [3]:
# hyper parameters
input_size = 784 # 28*28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [4]:
# Fully coneected neural network with one hidden layer
class LitNeuralNet(pl.LightningModule):
  def __init__(self, input_size, hidden_size, num_classes):
    super(LitNeuralNet, self).__init__()
    self.input_size = input_size
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    # no activation and no softmax at the end
    return out


  # ---training step---
  def training_step(self, batch, batch_idx):
    images, labels = batch
    images = images.reshape(-1, 28 * 28)

    # Forward pass
    outputs = self(images)
    loss = F.cross_entropy(outputs, labels)

    return {'loss': loss}

  def train_dataloader(self):
    # MNIST dataset
    train_dataset = torchvision.datasets.MNIST(
        root='/content/drive/MyDrive/study_DeepLearning/data/MNIST', train=True, transform=transforms.ToTensor(), download=True)
    # Data loader
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=batch_size, num_workers=2, shuffle=False
    )
    return train_loader
  # -------


  # ---validation step (for parameter tuning)---
  def val_dataloader(self):
    val_dataset = torchvision.datasets.MNIST(
        root='/content/drive/MyDrive/study_DeepLearning/data/MNIST', train=False, transform=transforms.ToTensor()
    )
    val_loader = torch.utils.data.DataLoader(
        dataset=val_dataset, batch_size=batch_size, num_workers=2, shuffle=False
    )
    return val_loader

  def validation_step(self, batch, batch_idx):
    images, labels = batch
    images = images.reshape(-1, 28 * 28)

    # Forward pass
    outputs = self(images)

    loss = F.cross_entropy(outputs, labels)
    return {"val_loss": loss}

  # def validation_epoch_end(self, outputs): <-- ver2.0からremoveされた※※※
  #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
  #   return {'val_loss': avg_loss}
  # -------

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=learning_rate)

In [10]:
# 試し
# trainer = Trainer(fast_dev_run=True) # fast_dev_run=Trueで1エポックのみ実行
# trainer = Trainer(max_epochs=num_epochs, fast_dev_run=False) # こっちで、全エポック学習
trainer = Trainer(accelerator="gpu", devices=1, max_epochs=num_epochs) # ※ GPUの指定方法！！！！

model = LitNeuralNet(input_size, hidden_size, num_classes)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 78.5 K
1 | relu | ReLU   | 0     
2 | l2   | Linear | 1.0 K 
--------------------------------
79.5 K    Trainable params
0         Non-trainable params
79.5 K    Total params
0.318     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
Tra